#### Functions for text formatting

In [115]:
RESET = "\033[0m"
BOLD = "\033[1m"
UNDERLINE = "\033[4m"
COLOR_RED = "\033[31m"
COLOR_GREEN = "\033[32m"
COLOR_CYAN = "\033[36m"

def textf(text, format):
    return f"{format}{text}{RESET}"

def bold(text):
    return textf(text, BOLD)

def underline(text):
    return textf(text, UNDERLINE)

# Anomaly Detection using K-Means and Spectral Clustering

## First: Importing the necessary libraries

In [146]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
import pickle

## Second: Importing the dataset and preprocessing it

- We first load the data from the CSVs.
- We then proceed with preprocessing the data as follows:
    1. Split the labels from the data (last column) and place them in separate dataframes.
    2. Encode the categorical data using into one-hot vectors using the OneHotEncoder class.
    3. Scale the features using the StandardScaler class.

In [121]:
def load_data() -> tuple:

    # Loading the data as indicated in the assignment
    # kddcup.data.corrected is the training data, corrected is the testing data
    print('Loading the data...', end=' ')
    training_data = pd.read_csv( 'archive/kddcup.data_10_percent_corrected', header=None )
    testing_data = pd.read_csv( 'archive/corrected', header=None )
    print( textf('Done!', COLOR_GREEN) )

    # Separating the labels (last column) from the features in the training and testing data
    print('Separating the labels from the features...', end=' ')
    X_train = training_data.iloc[:, :-1]
    y_train = training_data.iloc[:, -1]

    X_test = testing_data.iloc[:, :-1]
    y_test = testing_data.iloc[:, -1]
    print( textf('Done!', COLOR_GREEN) )

    # Concatenating the training and testing data vertically to perform OneHotEncoding on all the categorical features
    print('Concatenating the training and testing data...', end=' ')
    X = pd.concat( [X_train, X_test], axis=0 )
    print( textf('Done!', COLOR_GREEN) )

    # Encoding the features into one-hot vectors using OneHotEncoder
    # ColumnTransformer is used to apply the OneHotEncoder to the second, third and fourth columns (categorical features)
    # The remainder is set to 'passthrough' to keep the other columns unchanged (already numerical)
    print('Encoding the features into one-hot vectors...', end=' ')
    ct = ColumnTransformer( [('one_hot_encoder', OneHotEncoder(), [1, 2, 3])], remainder='passthrough' )
    ct = ct.fit(X)
    X_train = pd.DataFrame( ct.transform(X_train) )
    X_test = pd.DataFrame( ct.transform(X_test) )
    print( textf('Done!', COLOR_GREEN) )

    # Feature Scaling since some features have a much higher range than others
    print('Feature Scaling...', end=' ')
    scaler = StandardScaler()
    X_train = pd.DataFrame( scaler.fit_transform( X_train ) )
    X_test = pd.DataFrame( scaler.transform( X_test ) )
    print( textf('Done!', COLOR_GREEN) )

    print('All done!')

    return X_train, y_train, X_test, y_test

# Randomly picking some samples from the training data to speed up the training process (for testing purposes only)
def sample_data(X_train, y_train, X_test, y_test, n_train:int=10000, n_test:int=1000) -> tuple:
    print('Sampling the data...', end=' ')
    X_train = X_train.sample(n_train, random_state= 42)
    y_train = y_train[X_train.index]

    X_test = X_test.sample(n_test, random_state= 42)
    y_test = y_test[X_test.index]
    print( textf('Done!', COLOR_GREEN) )
    return X_train, y_train, X_test, y_test

In [125]:
X_train, y_train, X_test, y_test = load_data()

Loading the data... Done!
Separating the labels from the features... Done!
Concatenating the training and testing data... Done!
Encoding the features into one-hot vectors... Done!
Feature Scaling... Done!
All done!


In [126]:
# OPTIONAL: Uncomment the following line to sample the data (for testing purposes only)

# X_train, y_train, X_test, y_test = sample_data(X_train, y_train, X_test, y_test)

In [127]:
try:
    print('Number of unique labels in the training data: ', len(y_train.unique()))
    print('-' * 50)
    print('Number of unique labels in the testing data : ', len(y_test.unique()))
except NameError:
    print( textf('NameError: y_train or y_test is not defined', COLOR_RED) )

Number of unique labels in the training data:  23
--------------------------------------------------
Number of unique labels in the testing data :  38


## Third: Applying K-Means and Normalized Cut

### Evaluation Metrics

In [263]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


# 1. Precision
def precision(cluster_indices, cluster_labels, y_test):
    total_precision = 0
    for i in range( len(cluster_indices) ):
        # Calculating the precision of the ith cluster
        precision = np.sum( cluster_labels[i] == y_test[cluster_indices[i]] ) / len(cluster_indices[i])
        total_precision += precision
            
    average_precision = total_precision / len(cluster_indices)
    return average_precision

# 2. Recall
def recall(cluster_indices, cluster_labels, y_test):
    total_recall = 0
    for i in range( len(cluster_indices) ):
        # Calculating the recall of the ith cluster
        recall = np.sum( cluster_labels[i] == y_test[cluster_indices[i]] ) / len(y_test[cluster_indices[i]])
        total_recall += recall
        
    average_recall = total_recall / len(cluster_indices)
    return average_recall

# 3. F1 Score
def F1_score(cluster_indices, cluster_labels, y_test):
    # Calculating the F1 score
    # Here, we will iterate through each cluster and calculate the F1 score of the cluster
    # We add the F1 score of the ith cluster to the total F1 score which will then be used to calculate the average F1 score
    total_F1_score = 0
    for i in range( len(cluster_indices) ):
        # Calculating the F1 score of the ith cluster
        precision = precision(cluster_indices, cluster_labels, y_test)
        recall = recall(cluster_indices, cluster_labels, y_test)
        F1_score = 2 * precision * recall / (precision + recall)
        total_F1_score += F1_score

    # Calculating the average F1 score
    average_F1_score = total_F1_score / len(cluster_indices)

    return average_F1_score
# 4. Conditional Entropy
def conditional_entropy(cluster_indices, cluster_labels, y_test):
    return True

# 5. Confusion Matrix
def confusion_matrix(cluster_indices, cluster_labels, y_test):
    y_pred = np.zeros(len(y_test))
    for i in range(len(cluster_indices)):
        y_pred[cluster_indices[i]] = cluster_labels[i]
    cm = confusion_matrix(y_test, y_pred)
    print(ConfusionMatrixDisplay(cm).plot())
    return cm

# 6. Classification Report (Precision, Recall, F1-Score, Support)
def classification_report(cluster_indices, cluster_labels, y_test):
    y_pred = np.zeros(len(y_test))
    for i in range(len(cluster_indices)):
        y_pred[cluster_indices[i]] = cluster_labels[i]
    print(classification_report(y_test, y_pred))
    return

### Supplementary Functions

In [149]:
# This function will be used to calculate the euclidean distance between two data points
def euclidean_distance( row, centroid, data ) -> float:
    distance = 0.0
    for column in data.columns:
        distance += (row[column] - centroid[column])**2
    return np.sqrt(distance)


# This function will be used to calculate the mean of the data points in a cluster.
def calculate_mean( data ) -> pd.DataFrame:
    mean = pd.DataFrame(columns=data.columns)
    for column in data.columns:
        mean[column] = [data[column].mean()]
    return mean

# This function saves the kmeans model state using pickle
def save_state_kmeans( k: int, centroids: pd.DataFrame, clusters: dict, cluster_indices: dict, cluster_labels: dict ):
    with open('checkpoints/centroids' + str(k) + '.pkl', 'wb') as file:
        pickle.dump(centroids, file)
    with open('checkpoints/clusters' + str(k) + '.pkl', 'wb') as file:
        pickle.dump(clusters, file)
    with open('checkpoints/cluster_indices' + str(k) + '.pkl', 'wb') as file:
        pickle.dump(cluster_indices, file)
    with open('checkpoints/cluster_labels' + str(k) + '.pkl', 'wb') as file:
        pickle.dump(cluster_labels, file)

# This function loads the kmeans model state using pickle
def load_state_kmeans( k: int ) -> tuple:
    with open('checkpoints/centroids' + str(k) + '.pkl', 'rb') as file:
        centroids = pd.DataFrame( pickle.load(file) )
    with open('checkpoints/clusters' + str(k) + '.pkl', 'rb') as file:
        clusters = dict( pickle.load(file) )
    with open('checkpoints/cluster_indices' + str(k) + '.pkl', 'rb') as file:
        cluster_indices = dict( pickle.load(file) )
    with open('checkpoints/cluster_labels' + str(k) + '.pkl', 'rb') as file:
        cluster_labels = dict( pickle.load(file) )
    return centroids, clusters, cluster_indices, cluster_labels

### K-Means Clustering Algorithm

#### Implementation

In [129]:
def kmeans_clustering( k, data, max_iterations:int=None, print_updates=False, initial_centroids=None ):
    
    # Initially, selecting k random data points as centroids
    # We will use the current time as the seed to make sure that we get different centroids each time we run the algorithm
    
    np.random.seed( 42 )

    if initial_centroids is None:
        centroids = data.sample( k, random_state= 42 )
    else:
        centroids = initial_centroids.copy()
        
    old_centroids = None

    # If the user doesn't specify the maximum number of iterations, we will set it to infinity (Loop until convergence)
    if max_iterations is None:
        max_iterations = np.inf

    itr = 1
    while( itr <= max_iterations ):

        # If the centroids do not change, we will stop the algorithm
        if centroids.equals( old_centroids ):
            break

        # Storing the old centroids to check if they change in the next iteration
        old_centroids = centroids.copy()

        if print_updates is True: print( underline(bold(' Iteration #' + str(itr) + ' ')) )

        # Container initialization for cluster data
        clusters = {} # clusters[i] will store the data points in the (i+1)th cluster
        cluster_indices = {} # cluster_indices[i] will store the indices of the data points in the (i+1)th cluster
        cluster_labels = {} # cluster_labels[i] will store the label of the (i+1)th cluster by majority voting
        for i in range(k):
            clusters[i] = []
            cluster_indices[i] = []

        # Iterating through each data point and assigning it to the closest cluster
        for index, row in data.iterrows():

            min_distance, closest_cluster_index = np.inf, -1
            
            # Iterating through each centroid to find the closest one
            for i in range(k):

                # Using our euclidean_distance function to calculate the distance as it handles both numerical and categorical data
                current_distance = euclidean_distance( row, centroids.iloc[i], data )

                # Check if the data point is closer to the ith centroid
                if current_distance < min_distance:
                    min_distance = current_distance
                    closest_cluster_index = i
            
            # Assigning the data point to the cluster with the closest centroid
            clusters[closest_cluster_index].append( row )
            cluster_indices[closest_cluster_index].append( index )

        # Updating the centroids.
        # We will use our calculate_mean function to calculate the mean of the data points in the cluster
        # because it handles both numerical and categorical data
        for i in range(k):

            # If the cluster is empty, we will not update the centroid
            if len(clusters[i]) == 0:
                continue
            else:
                centroids.iloc[i] = calculate_mean( pd.DataFrame(clusters[i]) )

        
        # Calculating the cluster labels by majority voting (if the cluster is not empty)
        for i in range(k):
            if len(clusters[i]) == 0:
                cluster_labels[i] = None
            else:
                cluster_labels[i] = pd.Series( [y_train[index] for index in cluster_indices[i]] ).value_counts().index[0]

        # Printing the cluster sizes and labels in a pandas table
        if print_updates is True:
            print( pd.DataFrame( [ [len(clusters[i]), cluster_labels[i]] for i in range(k) ], columns=['Cluster Size', 'Cluster Label'] ) )

        if print_updates is True: print('-' * 50) # Just to print a line to separate the iterations

        save_state_kmeans( k, centroids, clusters, cluster_indices, cluster_labels )

        itr += 1

    return centroids, clusters, cluster_indices, cluster_labels

#### Execution

In [155]:
# This function will be used to calculate the purity of the clusters
# Purity is the percentage of data points in a cluster that belong to the same class
def calculate_purity( clusters, labels, print_report=False ):
    purities = []
    for i in range( len(clusters) ):
        cluster = clusters[i]

        # If the cluster is empty, we will skip it
        if len(cluster) == 0: continue

        # Converting the cluster to a dataframe so that we can use the value_counts() function
        cluster = pd.DataFrame(cluster)
        cluster['label'] = labels[cluster.index]

        # We will use the value_counts() function to count the number of data points in each class
        # and then we will divide it by the total number of data points in the cluster
        purities.append( cluster['label'].value_counts()[0] / len(cluster) )

    # Normalizing the purity by dividing it by the number of clusters
    average_purity = sum(purities) / len(clusters)

    if print_report is True:
        for i in range(len(purities)):
            print('Cluster ', i+1, ' purity: ', purities[i])
        print('-'*50)
        print('Average Purity: ', average_purity)
        print('-'*50)
    
    return average_purity, purities


# This function prints a report of the clusters produced by the k-means algorithm
def analyze_clusters( clusters, cluster_indices, cluster_labels, labels ):

    # Printing the number of data points in each cluster
    for i in range(len(cluster_indices)):
        print('Cluster ', i+1, ' contains ', len(cluster_indices[i]), ' data points of class ', cluster_labels[i])
    print( '-' * 50 )

    # Calculating the purity of the clusters and printing the report
    calculate_purity( clusters, labels, print_report=True )

    # Printing the count for each unique labels in each cluster horizontally
    for i in range(len(cluster_indices)):
        print( bold('[Cluster #' + str(i+1) + ']'), ' --> ', textf(cluster_labels[i], COLOR_CYAN) )
        print(labels[cluster_indices[i]].value_counts())
        print('-' * 50)

In [ ]:
# OPTIONAL: Uncomment the following line to load the saved state
# centroids7, clusters7, cluster_indices7, cluster_labels7 = load_state_kmeans( 7 )

In [132]:
try:
    centroids7, clusters7, cluster_indices7, cluster_labels7 = kmeans_clustering( k=7, data=X_train, print_updates=True )
except KeyboardInterrupt:
    print( textf('Process interrupted by user', COLOR_RED) )

 Iteration #1 
   Cluster Size Cluster Label
0        300767        smurf.
1             0          None
2             0          None
3         83600       normal.
4         92831      neptune.
5             0          None
6         16823      neptune.
--------------------------------------------------
 Iteration #2 
   Cluster Size Cluster Label
0         18092       normal.
1        280256        smurf.
2             0          None
3         84812       normal.
4         25048      neptune.
5             0          None
6         85813      neptune.
--------------------------------------------------
 Iteration #3 
   Cluster Size Cluster Label
0         21011       normal.
1         27685        smurf.
2        253586        smurf.
3         76526       normal.
4         28326      neptune.
5             0          None
6         86887      neptune.
--------------------------------------------------
 Iteration #4 
   Cluster Size Cluster Label
0         20987       normal.
1      

In [156]:
try:
    analyze_clusters( clusters7, cluster_indices7, cluster_labels7, y_train )
except NameError:
    print( textf('NameError: undefined arguments in analyze_clusters()', COLOR_RED) )

Cluster  1  contains  20991  data points of class  normal.
Cluster  2  contains  1688  data points of class  ipsweep.
Cluster  3  contains  11128  data points of class  smurf.
Cluster  4  contains  74760  data points of class  normal.
Cluster  5  contains  28218  data points of class  neptune.
Cluster  6  contains  270179  data points of class  smurf.
Cluster  7  contains  87057  data points of class  neptune.
--------------------------------------------------
Cluster  1  purity:  0.9417845743413844
Cluster  2  purity:  0.6824644549763034
Cluster  3  purity:  0.9535406182602444
Cluster  4  purity:  0.9544141252006421
Cluster  5  purity:  0.7249627897086965
Cluster  6  purity:  1.0
Cluster  7  purity:  0.9963816809676419
--------------------------------------------------
Average Purity:  0.8933640347792732
--------------------------------------------------
[Cluster #1]  -->  normal.
normal.         19769
teardrop.         979
satan.            172
nmap.              25
portsweep.       

In [ ]:
try:
    centroids15, clusters15, cluster_indices15, cluster_labels15 = kmeans_clustering( k=15, data=X_test, print_updates=True )
except KeyboardInterrupt:
    print( textf('Process interrupted by user', COLOR_RED) )

In [ ]:
try:
    analyze_clusters( clusters15, cluster_indices15, cluster_labels15, y_test )
except NameError:
    print( textf('NameError: undefined arguments in analyze_clusters()', COLOR_RED) )

In [ ]:
try:
    centroids23, clusters23, cluster_indices23, cluster_labels23 = kmeans_clustering( k=23, data=X_test, print_updates=True )
except KeyboardInterrupt:
    print( textf('Process interrupted by user', COLOR_RED) )

In [ ]:
try:
    analyze_clusters( clusters23, cluster_indices23, cluster_labels23, y_test )
except NameError:
    print( textf('NameError: undefined arguments in analyze_clusters()', COLOR_RED) )

In [ ]:
try:
    centroids31, clusters31, cluster_indices31, cluster_labels31 = kmeans_clustering( k=31, data=X_test, print_updates=True )
except KeyboardInterrupt:
    print( textf('Process interrupted by user', COLOR_RED) )

In [ ]:
try:
    analyze_clusters( clusters31, cluster_indices31, cluster_labels31, y_test )
except NameError:
    print( textf('NameError: undefined arguments in analyze_clusters()', COLOR_RED) )

In [ ]:
try:
    centroids45, clusters45, cluster_indices45, cluster_labels45 = kmeans_clustering( k=31, data=X_test, print_updates=True )
except KeyboardInterrupt:
    print( textf('Process interrupted by user', COLOR_RED) )

In [ ]:
try:
    analyze_clusters( clusters45, cluster_indices45, cluster_labels45, y_test )
except NameError:
    print( textf('NameError: undefined arguments in analyze_clusters()', COLOR_RED) )

### Normalized Cut Algorithm

#### Implementation (Not vectorized)

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
def constructWeightMatrix(data, gamma = 1):

    # Calculating the distance matrix
    distance_matrix = euclidean_distances( data, data )

    # Calculating the weight matrix
    weight_matrix = np.exp( -gamma * (distance_matrix**2) )

    # Returning the weight matrix
    return weight_matrix


def measure_cut( weight_matrix, clusters ):
    # Calculating the cut
    # Here, we will iterate through each pair of clusters and calculate the sum of the weights of the edges between them
    # We add the cut between the ith and jth clusters to the total cut measure which will then be used to calculate the normalized cut
    total_cut_measure = 0
    for i in range( len(clusters) ):
        for j in range( len(clusters) ):

            # We don't want to calculate the cut between a cluster and itself
            # So, we will skip the iteration if i == j
            if i == j: continue

            # Calculating the cut between the ith and jth clusters
            cut = np.sum( weight_matrix[clusters[i], :][:, clusters[j]] )
            total_cut_measure += cut
            
    # Calculating the volume
    # Here, we will iterate through each cluster and calculate the sum of the weights of the edges inside the cluster
    total_volume = 0
    for i in range( len(clusters) ):
        total_volume += np.sum( weight_matrix[clusters[i], :][:, clusters[i]] )

    # Calculating the normalized cut
    normalized_cut = total_cut_measure / total_volume

    return normalized_cut, total_cut_measure

def clusteringUsingNormalizedCut(data, k, gamma = 1):

    # Constructing the weight matrix
    weight_matrix = constructWeightMatrix(data, gamma=gamma)

    # Computing the degree matrix
    degree_matrix = np.diag( np.sum(weight_matrix, axis=1) )

    # Computing the laplacian matrix
    laplacian_matrix = degree_matrix - weight_matrix

    # Computing the eigenvalues and eigenvectors of the laplacian matrix
    eigenvalues, eigenvectors = np.linalg.eig(np.dot(np.linalg.inv(degree_matrix), laplacian_matrix))
    
    # Taking the only first k eigenvectors
    eigenvectors = eigenvectors[:, :k]

    # Applying k-means on the eigenvectors
    centroids, clusters, cluster_indices, cluster_labels = kmeans_clustering(k=k, data=eigenvectors, print_updates=False)
    
    # Evaluating the clusters using the normalized cut
    normalized_cut, total_cut_measure = measure_cut(weight_matrix, clusters)
    
    return centroids, clusters, cluster_indices, cluster_labels, normalized_cut, total_cut_measure

### Hierarchical Clustering (Agglomerative Clustering not Divisive)

In [107]:
from scipy.spatial import distance_matrix as distanceMatrix

In [255]:
def linkage(distance_matrix, clusters, cluster1, cluster2, linkage):
    if linkage == 'single':
        min_distance = np.inf
        for i in clusters[cluster1]:
            for j in clusters[cluster2]:
                if distance_matrix[i,j] < min_distance:
                    min_distance = distance_matrix[i,j]
        return min_distance
    elif linkage == 'complete':
        max_distance = -np.inf
        for i in clusters[cluster1]:
            for j in clusters[cluster2]:
                if distance_matrix[i,j] > max_distance:
                    max_distance = distance_matrix[i,j]
        return max_distance
    elif linkage == 'average' or linkage == 'centroid' or linkage == 'mean':
        sum_distance = 0
        for i in clusters[cluster1]:
            for j in clusters[cluster2]:
                sum_distance += distance_matrix[i,j]
        return sum_distance/(len(clusters[cluster2]) * len(clusters[cluster1]))
    else:
        raise ValueError('Invalid linkage type')
    
def hierarchical_clustering(data, linkage_type, n_clusters):
    
    # Marking each data point as a cluster
    clusters = [[i] for i in range(len(data))]

    # Calculating the distance matrix using Euclidean distance (default of distanceMatrix function)
    distance_matrix = distanceMatrix(data, data)

    # Iteratively merging the clusters
    while len(clusters) > n_clusters:

        # Finding closest two clusters
        min_distance = np.inf
        min_i,min_j = -1,-1
        for i in range(len(clusters)):
            for j in range(i+1,len(clusters)):
                dist = distance_matrix[i,j]
                if dist < min_distance:
                    min_distance = dist
                    min_i,min_j = i,j

        # Merging the closest two clusters
        clusters[min_i] = clusters[min_i] + clusters[min_j]

        # Removing the second cluster
        clusters.pop(min_j)
        
        # Adjusting the distance matrix 
        for i in range(len(clusters)):
            if i == min_i: continue
            distance_matrix[min_i,i] = distance_matrix[i,min_i] = linkage(distance_matrix, clusters, i, min_i, linkage_type)

    return clusters


In [260]:
# Testing the hierarchical clustering function
data = [[1,2],[3,4],[5,6],[7,8],[9,10],[11,12],[13,14],[15,16],[17,18],[19,20]]
clusters = hierarchical_clustering(data, 'mean', 5)
print(clusters)

[[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
